# The SO4 DMRG

The bilinear-biquadratic Hamiltonian is given by

$$
H_{b b}=\sum_i\left[J \sum_{a<b} L_i^{a b} L_{i+1}^{a b}+K\left(\sum_{a<b} L_i^{a b} L_{i+1}^{a b}\right)^2\right]
$$

For $n=4$ case, we can factorized it as $SO(4) \simeq SU(2)\times SU(2)$. Thus we can consider a spin-orbital $S=T=1/2$ two-leg spin ladder to implement the $SO(4)$ vectors and generators. We introduce

$$
\left|n^{1,2}\right\rangle=\frac{e^{ \pm i \pi / 4}}{\sqrt{2}}(|\uparrow, \uparrow\rangle \mp|\downarrow, \downarrow\rangle),
$$
and
$$
\left|n^{3,4}\right\rangle=\frac{e^{\mp i \pi / 4}}{\sqrt{2}}(|\downarrow, \uparrow\rangle \mp|\uparrow, \downarrow\rangle)
$$
where the $\ket{\sigma,\tau}$ denotes the spin and orbital directions. Moreover, the $SO(4)$ generators are defined by

$$
\begin{aligned}
& L^{12}=-T^z-S^z, \quad L^{13}=T^x-S^x, \quad L^{14}=-T^y-S^y \\
& L^{23}=T^y-S^y, \quad L^{24}=T^x+S^x, \quad L^{34}=T^z-S^z
\end{aligned}
$$

Which is a complex representation. 

We will have biquadratic terms in the Hamiltonian, following the spirit of $SO(6)$, we will finally have $4^2 = 16$ operators to represent both the bilinear and biquadratic terms. 

In [12]:
import numpy as np
import numpy.linalg as LA

def get_so4_opr_list():
    sigmax = np.array([[0, 1], [1, 0]])
    sigmay = np.array([[0, -1j], [1j, 0]])
    sigmaz = np.array([[1, 0], [0, -1]])
    id = np.eye(2)

    Sx = 0.5 * sigmax
    Sy = 0.5 * sigmay
    Sz = 0.5 * sigmaz
    Sp = Sx + 1j * Sy
    Sm = Sx - 1j * Sy

    L1 = -np.kron(Sz, id) - np.kron(id, Sz)
    L2 = -np.kron(Sx, id) + np.kron(id, Sx)
    L3 = -np.kron(Sy, id) - np.kron(id, Sy)
    L4 = -np.kron(Sy, id) + np.kron(id, Sy)
    L5 = +np.kron(Sx, id) + np.kron(id, Sx)
    L6 = -np.kron(Sz, id) + np.kron(id, Sz)

    Loprs = [L1, L2, L3, L4, L5, L6]
    coe_list = []

    for a in range(6):
        for b in range(6):
            LiLi = Loprs[a] @ Loprs[b]
            Amat = np.zeros((16, len(Loprs)), dtype=complex)
            B = LiLi.reshape(-1,1)
            for l in range(len(Loprs)):
                Amat[:,l] = Loprs[l].reshape(-1,1)[:,0]
            pcoe, resi, rank, sing = LA.lstsq(Amat, B, rcond=None)
            if len(resi)!=0 and resi[0]>1e-10:
                Loprs.append(LiLi)
                pcoe = np.append(np.zeros((len(Loprs)-1, 1)),1).reshape(len(Loprs),1)
                coe_list.append(pcoe)
            else:
                coe_list.append(pcoe)

    coe_list_new = []
    for a in range(6):
        for b in range(6):
            LiLi = Loprs[a] @ Loprs[b]
            Amat = np.zeros((16, len(Loprs)), dtype=complex)
            B = LiLi.reshape(-1,1)
            for l in range(len(Loprs)):
                Amat[:,l] = Loprs[l].reshape(-1,1)[:,0]
            pcoe = LA.solve(Amat, B)
            coe_list_new.append(pcoe)
    
    coe_list = coe_list_new

    for i in range(len(coe_list)):
        coe_list[i] = coe_list[i].reshape(16)

    def pvec(a,b):
        return coe_list[6*a+b] #a,b=0,1,2,3,4,5
    
    cmn = np.zeros((16,16), dtype=complex)

    P = dict()
    for a in range(6):
        for b in range(6):
            P[(a,b)] = pvec(a,b)
    
    for m in range(16):
        for n in range(16):
            for a in range(6):
                for b in range(6):
                    cmn[m,n] += P[(a,b)][m] * P[(a,b)][n]

    return Loprs, cmn

In [13]:
L_operators, c_mn = get_so4_opr_list()

for i in range(len(L_operators)):
    print("the", i, "th operator is")
    print(L_operators[i])
    print('')

the 0 th operator is
[[-1. -0. -0. -0.]
 [-0.  0. -0.  0.]
 [-0. -0.  0.  0.]
 [-0.  0.  0.  1.]]

the 1 th operator is
[[ 0.   0.5 -0.5  0. ]
 [ 0.5  0.   0.  -0.5]
 [-0.5  0.   0.   0.5]
 [ 0.  -0.5  0.5  0. ]]

the 2 th operator is
[[-0.-0.j  -0.+0.5j -0.+0.5j -0.-0.j ]
 [-0.-0.5j -0.-0.j  -0.-0.j  -0.+0.5j]
 [-0.-0.5j -0.-0.j  -0.-0.j  -0.+0.5j]
 [-0.-0.j  -0.-0.5j -0.-0.5j -0.-0.j ]]

the 3 th operator is
[[0.+0.j  0.-0.5j 0.+0.5j 0.+0.j ]
 [0.+0.5j 0.+0.j  0.+0.j  0.+0.5j]
 [0.-0.5j 0.+0.j  0.+0.j  0.-0.5j]
 [0.+0.j  0.-0.5j 0.+0.5j 0.+0.j ]]

the 4 th operator is
[[0.  0.5 0.5 0. ]
 [0.5 0.  0.  0.5]
 [0.5 0.  0.  0.5]
 [0.  0.5 0.5 0. ]]

the 5 th operator is
[[ 0.  0.  0.  0.]
 [ 0. -1.  0. -0.]
 [ 0.  0.  1.  0.]
 [ 0. -0.  0.  0.]]

the 6 th operator is
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]

the 7 th operator is
[[ 0.  -0.5  0.5  0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.  -0.5  0.5  0. ]]

the 8 th operator is
[[0.+0.j  0.-0.5j 0.-0.5j 0

Then we are going to make the biquadratic terms represented by these 16 on-site operators. That is the expansion

$$
\left(\sum_{a,b=1}^6 L^a(i) \otimes L^b(i+1)\right) = \sum_{m,n=1}^{16} c_{mn} L^m(i)\otimes L^n(i+1)
$$

In the latest python block, we have this decomposition done
$$
L^a(i)L^b(i) = \sum_{m=1}^{16} p_m(a,b) L^m(i)
$$

Then we have
$$
L^a(i)L^b(i) \otimes L^a(i+1)L^b(i+1) = \sum_{m=1}^{16}\sum_{n=1}^{16} p_m(a,b)p_n(a,b) L^m(i) \otimes L^n(i+1)
$$

By defining 
$$
c_{mn} = \sum_{a,b=1}^6 p_m(a,b) p_n(a,b), 
$$
everything follow the $SO(6)$ case. 

There are 16 on-site operators, they are

$$
L^0=\left(\begin{array}{cccc}
-1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1
\end{array}\right), \quad 
L^1=\frac{1}{2}\left(\begin{array}{cccc}
0 & 1 & -1 & 0\\
1 & 0 & 0 & -1 \\
-1 & 0 & 0 & 1 \\
0 & -1 & 1 & 0 
\end{array}\right), \quad
L^2=\frac{1}{2}\left(\begin{array}{cccc}
0 & i & i & 0\\
-i & 0 & 0 & i \\
-i & 0 & 0 & i \\
0 & -i & -i & 0 
\end{array}\right), \quad
L^3=\frac{1}{2}\left(\begin{array}{cccc}
0 & -i & i & 0\\
i & 0 & 0 & i \\
-i & 0 & 0 & -i \\
0 & -i & i & 0 
\end{array}\right),
$$

$$
L^4=\frac{1}{2}\left(\begin{array}{cccc}
0 & 1 & 1 & 0\\
1 & 0 & 0 & 1 \\
1 & 0 & 0 & 1 \\
0 & 1 & 1 & 0 
\end{array}\right), \quad 
L^5\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0
\end{array}\right), \quad
L^6=\left(\begin{array}{cccc}
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1
\end{array}\right), \quad
L^7=\frac{1}{2}\left(\begin{array}{cccc}
0 & -1 & 1 & 0\\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & -1 & 1 & 0 
\end{array}\right),
$$

$$
L^8=\frac{1}{2}\left(\begin{array}{cccc}
0 & -i & -i & 0\\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & -i & -i & 0 
\end{array}\right), \quad 
L^9=\frac{1}{2}\left(\begin{array}{cccc}
0 & i & -i & 0\\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & -i & i & 0 
\end{array}\right), \quad
L^{10}=\frac{1}{2}\left(\begin{array}{cccc}
0 & -1 & -1 & 0\\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 1 & 1 & 0 
\end{array}\right), \quad
L^{11}=\frac{1}{2}\left(\begin{array}{cccc}
1 & 0 & 0 & -1\\
0 & 1 & -1 & 0 \\
0 & -1 & 1 & 0 \\
-1 & 0 & 0 & 1 
\end{array}\right),
$$

$$
L^{12}=\frac{1}{2}\left(\begin{array}{cccc}
0 & 0 & 0 & 0\\
0 & i & i & 0 \\
0 & -i & -i & 0 \\
0 & 0 & 0 & 0 
\end{array}\right), \quad 
L^{13}=\frac{1}{2}\left(\begin{array}{cccc}
i & 0 & 0 & i\\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
-i & 0 & 0 & -i 
\end{array}\right), \quad
L^{14}=\frac{1}{2}\left(\begin{array}{cccc}
1 & 0 & 0 & -1\\
0 & 1 & 1 & 0 \\
0 & 1 & 1 & 0 \\
-1 & 0 & 0 & 1 
\end{array}\right), \quad
L^{15}=\frac{1}{2}\left(\begin{array}{cccc}
1 & 0 & 0 & 1\\
0 & 1 & -1 & 0 \\
0 & -1 & 1 & 0 \\
1 & 0 & 0 & 1 
\end{array}\right),
$$

We notice that there are 2 diagonal matrices in the $SO(4)$ generators, they are

$$
L^0=\left(\begin{array}{cccc}
-1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1
\end{array}\right), \quad 
L^5\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0
\end{array}\right)
$$

The Cartan sub-algebra shows up automatically, it is 2-dimensional and we are allowed to have $U(1)\times U(1)$ good quantum numbers. But we can not use the symmetry directly, obviously the on-site operators can not be converted into charge conserved matrices under the $U(1)\times U(1)$ good quantum numbers. 

Under the standard basis $\{ \ket{\uparrow \uparrow}, \ket{\uparrow \downarrow}, \ket{\downarrow \uparrow}, \ket{\downarrow \downarrow} \}$, a possible set of leg charges is

```chinfo = npc.ChargeInfo([1, 1], ['S', 'T'])```

```leg = npc.LegCharge.from_qflat(chinfo, [[-1, 0], [0, -1], [0, 1], [1, 0]])```

With this leg, the $4\times 4$ matrix form of any operator holds the $U(1)\times U(1)$ symmetry should have total quantum numbers below
$$
\begin{array}{|cc|cc|cc|cc|}
\hline 0 & 0 & -1 & 1 & -1 & -1 & -2 & 0 \\
\hline 1 & -1 & 0 & 0 & 0 & -2 & -1 & -1 \\
\hline 1 & 1 & 0 & 2 & 0 & 0 & -1 & 1 \\
\hline 2 & 0 & 1 & 1 & 1 & -1 & 0 & 0 \\
\hline
\end{array}
$$

Then we are going to do some linear combinations in order to make the $L$ operators fit in the leg charges. 

$$
\hat{L}^0=L^0
=\left(\begin{array}{cccc}
-1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1
\end{array}\right)
$$

$$
\hat{L}^1
=\frac{1}{2} \left[\left(L^1 + L^4\right) - i\left(L^2-L^3\right)\right]
=\left(\begin{array}{cccc}
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0
\end{array}\right)
$$

$$
\hat{L}^2
=\frac{1}{2} \left[\left(L^1 + L^4\right) + i\left(L^2-L^3\right)\right]
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0
\end{array}\right)
$$

$$
\hat{L}^3
=\frac{1}{2} \left[\left(L^1 - L^4\right) - i\left(L^2+L^3\right)\right]
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
-1 & 0 & 0 & 0 \\
0 & -1 & 0 & 0
\end{array}\right)
$$

$$
\hat{L}^4
=\frac{1}{2} \left[\left(L^1 - L^4\right) + i\left(L^2+L^3\right)\right]
=\left(\begin{array}{cccc}
0 & 0 & -1 & 0 \\
0 & 0 & 0 & -1 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{array}\right)
$$

$$
\hat{L}^5=L^5
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0
\end{array}\right)
$$

In [14]:
print('Lhat[1]', 1/2 * ((L_operators[1]+L_operators[4]) - 1j*(L_operators[2]-L_operators[3])))

print('Lhat[2]', 1/2 * ((L_operators[1]+L_operators[4]) + 1j*(L_operators[2]-L_operators[3])))

print('Lhat[3]', 1/2 * ((L_operators[1]-L_operators[4]) - 1j*(L_operators[2]+L_operators[3])))

print('Lhat[4]', 1/2 * ((L_operators[1]-L_operators[4]) + 1j*(L_operators[2]+L_operators[3])))

Lhat[1] [[0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
Lhat[2] [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]]
Lhat[3] [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]]
Lhat[4] [[ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]]


$$
\hat{L}^6=L^6
=\left(\begin{array}{cccc}
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1
\end{array}\right)
$$

$$
\hat{L}^7
=-\frac{1}{2} \left[\left(L^7 + L^{10}\right) - i\left(L^8-L^9\right)\right]
=\left(\begin{array}{cccc}
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{array}\right)
$$

$$
\hat{L}^8
=\frac{1}{2} \left[\left(L^7 + L^{10}\right) + i\left(L^8-L^9\right)\right]
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0
\end{array}\right)
$$

$$
\hat{L}^9
=-\frac{1}{2} \left[\left(L^7 - L^{10}\right) - i\left(L^8+L^9\right)\right]
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0
\end{array}\right)
$$

$$
\hat{L}^{10}
=\frac{1}{2} \left[\left(L^7 - L^{10}\right) + i\left(L^8+L^9\right)\right]
=\left(\begin{array}{cccc}
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{array}\right)
$$

In [15]:
print('Lhat[7]', -1/2 * ((L_operators[7]+L_operators[10]) - 1j*(L_operators[8]-L_operators[9])))

print('Lhat[8]', 1/2 * ((L_operators[7]+L_operators[10]) + 1j*(L_operators[8]-L_operators[9])))

print('Lhat[9]', -1/2 * ((L_operators[7]-L_operators[10]) - 1j*(L_operators[8]+L_operators[9])))

print('Lhat[10]', 1/2 * ((L_operators[7]-L_operators[10]) + 1j*(L_operators[8]+L_operators[9])))

Lhat[7] [[-0.+0.j  1.-0.j -0.+0.j -0.+0.j]
 [-0.+0.j -0.+0.j -0.+0.j -0.+0.j]
 [-0.+0.j -0.+0.j -0.+0.j -0.+0.j]
 [-0.+0.j -0.+0.j -0.+0.j -0.+0.j]]
Lhat[8] [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]]
Lhat[9] [[-0.+0.j -0.+0.j -0.+0.j -0.+0.j]
 [-0.+0.j -0.+0.j -0.+0.j -0.+0.j]
 [-0.+0.j -0.+0.j -0.+0.j -0.+0.j]
 [-0.+0.j  1.-0.j -0.+0.j -0.+0.j]]
Lhat[10] [[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


$$
\hat{L}^{11}=\frac{1}{2}\left(2i L^{13} - L^{0} + L^{15}-L^{11}\right)
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0
\end{array}\right)
$$

$$
\hat{L}^{12}=\frac{1}{2}\left(-2i L^{13} + L^{0} + L^{15}-L^{11}\right)
=\left(\begin{array}{cccc}
0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{array}\right)
$$

$$
\hat{L}^{13}=\frac{1}{2}\left(2i L^{12} - L^{5} + L^{14}-L^{11}\right)
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0
\end{array}\right)
$$

$$
\hat{L}^{14}=\frac{1}{2}\left(-2i L^{12} + L^{5} + L^{14}-L^{11}\right)
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{array}\right)
$$

In [16]:
print('Lhat[11]', 2j*L_operators[13]-L_operators[0]+L_operators[15]-L_operators[11])

print('Lhat[12]', -2j*L_operators[13]+L_operators[0]+L_operators[15]-L_operators[11])

print('Lhat[13]', 2j*L_operators[12]-L_operators[5]+L_operators[14]-L_operators[11])

print('Lhat[14]', -2j*L_operators[12]+L_operators[5]+L_operators[14]-L_operators[11])

Lhat[11] [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [2.+0.j 0.+0.j 0.+0.j 0.+0.j]]
Lhat[12] [[0.+0.j 0.+0.j 0.+0.j 2.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
Lhat[13] [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 2.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
Lhat[14] [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 2.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


With the identity matrix $\hat{L}^{15}=L^{14}+L^{15}$, we have totally 16 new matrices equipped with $U(1)\times U(1)$ leg charges. They are

$$
\hat{L}^0
=\left(\begin{array}{cccc}
-1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1
\end{array}\right), \quad

\hat{L}^1
=\left(\begin{array}{cccc}
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0
\end{array}\right), \quad

\hat{L}^2
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0
\end{array}\right), \quad

\hat{L}^3
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
-1 & 0 & 0 & 0 \\
0 & -1 & 0 & 0
\end{array}\right), \quad
$$

$$
\hat{L}^4
=\left(\begin{array}{cccc}
0 & 0 & -1 & 0 \\
0 & 0 & 0 & -1 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{array}\right), \quad

\hat{L}^5
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0
\end{array}\right), \quad

\hat{L}^6
=\left(\begin{array}{cccc}
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1
\end{array}\right), \quad

\hat{L}^7
=\left(\begin{array}{cccc}
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{array}\right), \quad
$$

$$
\hat{L}^8
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0
\end{array}\right), \quad

\hat{L}^9
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0
\end{array}\right), \quad

\hat{L}^{10}
=\left(\begin{array}{cccc}
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{array}\right), \quad

\hat{L}^{11}
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0
\end{array}\right), \quad
$$

$$
\hat{L}^{12}
=\left(\begin{array}{cccc}
0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{array}\right), \quad

\hat{L}^{13}
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0
\end{array}\right), \quad

\hat{L}^{14}
=\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{array}\right), \quad

\hat{L}^{15}
=\left(\begin{array}{cccc}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{array}\right), \quad
$$

Now we should make the biquadratic terms represented by the new hatted 16 on-site operators. That is the expansion

$$
\left(\sum_{a,b=1}^6 L^a(i) \otimes L^b(i+1)\right) = \sum_{m,n=0}^{15} c_{mn} \hat{L}^m(i)\otimes \hat{L}^n(i+1)
$$

That is to say, we should get this decomposition done again
$$
L^a(i)L^b(i) = \sum_{m=0}^{15} p_m(a,b) \hat{L}^m(i)
$$

Then we will have
$$
L^a(i)L^b(i) \otimes L^a(i+1)L^b(i+1) = \sum_{m=0}^{15}\sum_{n=0}^{15} p_m(a,b)p_n(a,b) \hat{L}^m(i) \otimes \hat{L}^n(i+1)
$$

By defining 
$$
c_{mn} = \sum_{a,b=1}^6 p_m(a,b) p_n(a,b), 
$$
everything follow the $SO(6)$ case again. 

(20241003) Notice that the bilinear terms are different from the SO(6) case since we have changed the whole basis. 

$$
\begin{aligned}
H_{\mathrm{bilinear}} &= \sum_{i=1}^N J \sum_{a=1}^6 L^a(i)\otimes L^a(i+1) \\
&= \sum_{i=1}^N J \sum_{a=1}^6 \left(\sum_{m=0}^{15} q_m(a)\hat{L}^m(i) \otimes \sum_{n=0}^{15} q_n(a)\hat{L}^n(i+1) \right) \\
&= \sum_{i=1}^N J \sum_{m,n=0}^{15} \sum_{a=1}^6 q_m(a) q_n(a) \hat{L}^m(i) \hat{L}^n(i+1) \\
&= \sum_{i=1}^N J \sum_{m,n=0}^{15} d_{mn} \hat{L}^m(i) \hat{L}^n(i+1)
\end{aligned}
$$

In [17]:
def get_so4_opr_list_new():
    sigmax = np.array([[0, 1], [1, 0]])
    sigmay = np.array([[0, -1j], [1j, 0]])
    sigmaz = np.array([[1, 0], [0, -1]])
    id = np.eye(2)

    Sx = 0.5 * sigmax
    Sy = 0.5 * sigmay
    Sz = 0.5 * sigmaz

    L1 = -np.kron(Sz, id) - np.kron(id, Sz)
    L2 = -np.kron(Sx, id) + np.kron(id, Sx)
    L3 = -np.kron(Sy, id) - np.kron(id, Sy)
    L4 = -np.kron(Sy, id) + np.kron(id, Sy)
    L5 = +np.kron(Sx, id) + np.kron(id, Sx)
    L6 = -np.kron(Sz, id) + np.kron(id, Sz)

    Loprs = [L1, L2, L3, L4, L5, L6]
    Lhatoprs = [
    np.array([
        [-1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1]
    ]),
    np.array([
        [0, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0]
    ]),
    np.array([
        [0, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 1, 0]
    ]),
    np.array([
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [-1, 0, 0, 0],
        [0, -1, 0, 0]
    ]),
    np.array([
        [0, 0, -1, 0],
        [0, 0, 0, -1],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ]),
    np.array([
        [0, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 0]
    ]),
    np.array([
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1]
    ]),
    np.array([
        [0, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ]),
    np.array([
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 1, 0]
    ]),
    np.array([
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 1, 0, 0]
    ]),
    np.array([
        [0, 0, 1, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ]),
    np.array([
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 0, 0, 0]
    ]),
    np.array([
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ]),
    np.array([
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 0]
    ]),
    np.array([
        [0, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ]),
    np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
]

    coe_list = []
    for a in range(6):
        for b in range(6):
            LiLi = Loprs[a] @ Loprs[b]
            Amat = np.zeros((16, len(Lhatoprs)), dtype=complex)
            B = LiLi.reshape(-1,1)
            for l in range(len(Lhatoprs)):
                Amat[:,l] = Lhatoprs[l].reshape(-1,1)[:,0]
            pcoe = LA.solve(Amat, B)
            coe_list.append(pcoe)

    for i in range(len(coe_list)):
        coe_list[i] = coe_list[i].reshape(16)

    def pvec(a,b):
        return coe_list[6*a+b] #a,b=0,1,2,3,4,5
    
    cmn = np.zeros((16,16), dtype=complex)

    P = dict()
    for a in range(6):
        for b in range(6):
            P[(a,b)] = pvec(a,b)
    
    for m in range(16):
        for n in range(16):
            for a in range(6):
                for b in range(6):
                    cmn[m,n] += P[(a,b)][m] * P[(a,b)][n]

    coe_list_new = []
    for a in range(6):
        Li = Loprs[a]
        Amat = np.zeros((16, len(Lhatoprs)), dtype=complex)
        B = Li.reshape(-1,1)
        for l in range(len(Lhatoprs)):
            Amat[:,l] = Lhatoprs[l].reshape(-1,1)[:,0]
        qcoe = LA.solve(Amat, B)
        coe_list_new.append(qcoe)
    
    for i in range(len(coe_list_new)):
        coe_list_new[i] = coe_list_new[i].reshape(16)

    def qvec(a):
        return coe_list_new[a]
    
    dmn = np.zeros((16,16), dtype=complex)
    
    Q = dict()
    for a in range(6):
        Q[a] = qvec(a)

    for m in range(16):
        for n in range(16):
            for a in range(6):
                dmn[m,n] += Q[a][m] * Q[a][n]

    return Lhatoprs, cmn, dmn

In [18]:
from tenpy.models.model import CouplingModel, MPOModel
from tenpy.networks.site import SpinSite, FermionSite, Site
from tenpy.models.lattice import Chain
from tenpy.networks.mps import MPS
from tenpy.networks.mpo import MPO
import tenpy.linalg.np_conserved as npc
from tenpy.linalg.charges import LegCharge, ChargeInfo
from tenpy.algorithms import dmrg
from tenpy.tools.params import asConfig

In [19]:
class SO4Site(Site):
    def __init__(self, so4g, cons_N=None, cons_S=None):
        self.conserve = [cons_N, cons_S]
        self.cons_N = cons_N
        self.cons_S = cons_S
        self.so4g = so4g
        
        if self.cons_N is None and self.cons_S == 'U1':
            chinfo = npc.ChargeInfo([1, 1], ['S', 'T'])
            leg = npc.LegCharge.from_qflat(chinfo, [[-1, 0], [0, -1], [0, 1], [1, 0]])
        elif self.cons_N is None and self.cons_S is None:
            leg = npc.LegCharge.from_trivial(4)
        
        ops = dict()
        for i in range(len(self.so4g)):
            ops['L{}'.format(i)] = self.so4g[i]

        names = ['1u2u', '1u2d', '1d2u', '1d2d']
        Site.__init__(self, leg, names, **ops)

    def __repr__(self):
        return "trivial site for 16 so4 generators"

In [20]:
class BBQJKSO4(CouplingModel):
    def __init__(self, model_params):
        print(model_params)
        model_params = asConfig(model_params, self.__class__.__name__)
        self.model_params = model_params
        self.Lx = model_params.get('Lx', 8)
        self.S = model_params.get('S', 1)
        self.bc = model_params.get('bc', 'periodic')
        self.J = model_params.get('J', 1)
        self.K = model_params.get('K', 1/4)
        self.verbose = model_params.get('verbose', 2)
        self.D = model_params.get('D', 200)
        self.sweeps = model_params.get('sweeps', 10)
        
        self.so4_generators, self.c_mn, self.d_mn = get_so4_opr_list_new()

        site = SO4Site(self.so4_generators, cons_N=None, cons_S='U1')
        self.sites = [site] * self.Lx
        self.lat = Chain(self.Lx, site, bc=self.bc)
        CouplingModel.__init__(self, self.lat, explicit_plus_hc=False)
        self.init_terms(model_params)
        H_MPO = self.calc_H_MPO()
        if model_params.get('sort_mpo_legs', False):
            H_MPO.sort_legcharges()
        MPOModel.__init__(self, self.lat, H_MPO)
        model_params.warn_unused()

    def init_terms(self, model_params):
        J = model_params.get("J", 1.)
        K = model_params.get('K', 1/4)
        for l in range(self.Lx):
            if l < self.Lx - 1:
                i0, i1 = l, (l+1)%self.Lx
            elif l == self.Lx-1 and self.bc == 'periodic':
                i0, i1 = 0, self.Lx-1
                print("periodic terms added")
            else:
                break
        
            for m in range(16):
                for n in range(16):
                    self.add_coupling_term(J*np.round(self.d_mn[m,n],6), i0, i1, "L"+str(m), "L"+str(n))
                    self.add_coupling_term(K*np.round(self.c_mn[m,n],6), i0, i1, "L"+str(m), "L"+str(n))
    
    def run_dmrg(self, **kwargs):
        mixer      = kwargs.get('mixer', True)
        chi_max    = kwargs.get('chi_max', self.D)
        max_E_err  = kwargs.get('max_E_err', 1e-10)
        max_sweeps = kwargs.get('max_sweeps', self.sweeps)
        min_sweeps = kwargs.get('min_sweeps', min(4, max_sweeps) )
        dmrg_params = dict(mixer=mixer, 
                           trunc_params=dict(chi_max=chi_max),
                           max_E_err=max_E_err, 
                           max_sweeps=max_sweeps,
                           min_sweeps=min_sweeps,
                           verbose=2)

        init = kwargs.get('init', None)
        if init is None:
            N = self.lat.N_sites
            if N%4==0 and N>0:
                init = [0]*(N//4) + [1]*(N//4) + [2]*(N//4) + [3]*(N//4)
            else:
                raise("Check the system size must be integral multiple of 6")
            np.random.shuffle(init)
            psiinit = MPS.from_product_state(self.lat.mps_sites(), init)
            psiinit.norm = 1
            psiinit.canonical_form()
        elif isinstance(init, list):
            psiinit = MPS.from_product_state(self.lat.mps_sites(), init)            
        elif isinstance(init, MPS):
            psiinit = init
        else:
            print("wrong init")

        eng = dmrg.TwoSiteDMRGEngine(psiinit, self, dmrg_params)
        E, psidmrg = eng.run()
        print("Eng = ", E)
        self.psidmrg = psidmrg
        return psidmrg, E

In [21]:
import logging
logging.basicConfig(level=2)
for _ in ['parso.python.diff', 'parso.cache', 'parso.python.diff', 
            'parso.cache', 'matplotlib.font_manager', 'tenpy.tools.cache', 
            'tenpy.algorithms.mps_common', 'tenpy.linalg.lanczos', 'tenpy.tools.params']:
    logging.getLogger(_).disabled = True

model_paras = dict(cons_N=None, cons_S=None, Lx = 12, bc='periodic', J=1., K=1/4, D=200, sweeps=10, verbose=2)
so4bbq = BBQJKSO4(model_paras)
print("----------Start Job DMRG----------")
psi_dmrg, E = so4bbq.run_dmrg()
print("DMRG results")
print("DMRG psi", psi_dmrg)

print("entropy", psi_dmrg.entanglement_entropy())

{'cons_N': None, 'cons_S': None, 'Lx': 12, 'bc': 'periodic', 'J': 1.0, 'K': 0.25, 'D': 200, 'sweeps': 10, 'verbose': 2}
periodic terms added


/opt/anaconda3/lib/python3.9/site-packages/tenpy/tools/params.py:237: UserWarning: unused options for config BBQJKSO4:
['cons_N', 'cons_S']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
INFO:tenpy.algorithms.dmrg:Running sweep with optimization
DEBUG:tenpy.linalg.krylov_based:Lanczos N=20, gap=1.137e-01, DeltaE0=5.306e-09, _result_krylov[-1]=3.539e-05
DEBUG:tenpy.linalg.krylov_based:Lanczos N=20, gap=6.491e-02, DeltaE0=3.047e-05, _result_krylov[-1]=-2.932e-03


----------Start Job DMRG----------


DEBUG:tenpy.linalg.krylov_based:Lanczos N=20, gap=7.482e-01, DeltaE0=2.809e-06, _result_krylov[-1]=7.853e-04
DEBUG:tenpy.linalg.krylov_based:Lanczos N=20, gap=7.966e-01, DeltaE0=1.791e-10, _result_krylov[-1]=-6.166e-06
DEBUG:tenpy.linalg.krylov_based:Lanczos N=20, gap=6.840e-01, DeltaE0=6.937e-11, _result_krylov[-1]=4.004e-06
DEBUG:tenpy.linalg.krylov_based:Lanczos N=20, gap=7.017e-01, DeltaE0=5.728e-10, _result_krylov[-1]=1.144e-05
DEBUG:tenpy.linalg.krylov_based:Lanczos N=20, gap=2.030e-01, DeltaE0=1.801e-06, _result_krylov[-1]=6.417e-04
DEBUG:tenpy.linalg.krylov_based:Lanczos N=20, gap=5.100e-01, DeltaE0=3.643e-08, _result_krylov[-1]=-1.037e-04
DEBUG:tenpy.linalg.krylov_based:Lanczos N=20, gap=4.246e-01, DeltaE0=4.566e-11, _result_krylov[-1]=3.177e-06
DEBUG:tenpy.linalg.krylov_based:Lanczos N=20, gap=2.531e-01, DeltaE0=3.844e-07, _result_krylov[-1]=2.566e-04
DEBUG:tenpy.linalg.krylov_based:Lanczos N=20, gap=1.693e-01, DeltaE0=2.915e-06, _result_krylov[-1]=6.888e-04
DEBUG:tenpy.linal

Eng =  -8.999999999999998
DMRG results
DMRG psi MPS, L=12, bc='finite'.
chi: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
first two sites: trivial site for 16 so4 generators trivial site for 16 so4 generators
first two forms:(1.0, 0.0) (1.0, 0.0)
entropy [1.38629436 1.38629436 1.38629436 1.38629436 1.38629436 1.38629436
 1.38629436 1.38629436 1.38629436 1.38629436 1.38629436]


/opt/anaconda3/lib/python3.9/site-packages/tenpy/tools/params.py:237: UserWarning: unused option ['verbose'] for config TwoSiteDMRGEngine

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
